In [1]:
import numpy as np
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import pickle

D:\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


This notebook is used to train word2vec embeddings with gensim over the given training and validation datasets.

In [3]:
#titles = pd.read_csv('df/train_df.csv')['title']
#texts = pd.read_csv('df/train_df.csv')['text']
titles = pd.read_csv('df/test_df.csv')['title']
texts = pd.read_csv('df/test_df.csv')['text']

In [4]:
sentences = []
for title in titles:
    if type(title) == str:
        sentences.append(word_tokenize(title))

In [5]:
total = len(texts)
for i in range(len(texts)):
    if i % 10000 == 0:
        print(i)
    text = texts[i]
    if type(text) == str:
        text = sent_tokenize(text)
        for sent in text:
            sentences.append(word_tokenize(sent))

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000


In [6]:
with open('sentences-test.pkl', 'wb') as f:
    pickle.dump(sentences, f)
with open('sentences.pkl', 'rb') as f:
    sentences = sentences + pickle.load(f)

In [7]:
embed_dim = 50

model = gensim.models.Word2Vec(sentences, 
                               size = embed_dim, 
                               window = 5, 
                               min_count = 5, 
                               negative = 15,
                               iter = 2, 
                               workers = 12)

In [16]:
model.most_similar(positive=['trump'])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('negate', 0.7446917295455933),
 ('taint', 0.7144226431846619),
 ('distort', 0.7076326012611389),
 ('invoke', 0.7037114500999451),
 ('signify', 0.6934577226638794),
 ('tarnish', 0.6929789185523987),
 ('supersede', 0.6926395297050476),
 ('eviscerate', 0.6918069124221802),
 ('eschew', 0.6872264742851257),
 ('impugn', 0.6792529821395874)]

In [10]:
word2index = {token: token_index for token_index, token in enumerate(model.wv.index2word)} 

In [11]:
pickle.dump(word2index, open("embed_total_idx.pkl", "wb"))

In [12]:
embedding_matrix = np.zeros((len(model.wv.vocab), embed_dim))
for i in range(len(model.wv.vocab)):
    embedding_vector = model[model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
np.save('embed_total_vec', embedding_matrix)

In [14]:
embedding_matrix.shape

(457196, 50)